<a href="https://colab.research.google.com/github/arunv8055/Soya-Market-Analysis/blob/main/Soya_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Essential Library for Soya Analysis

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Importing and Preparing the dataset

In [33]:
url = 'https://raw.githubusercontent.com/arunv8055/Soya-Market-Analysis/main/Soya_August_Price.csv'
df = pd.read_csv(url)
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date
0,Andhra Pradesh,Kurnool,Kurnool,Soyabeen,Oil Seeds,0.1,2509,4610,2509,27-Jul-24
1,Chattisgarh,Balodabazar,Bhatapara,Other,Oil Seeds,0.4,4200,4281,4281,26-Sep-24
2,Chattisgarh,Kabirdham,Biranpur kalan (Sahaspur Lohra),Yellow,Oil Seeds,340,4050,4050,4050,2-Aug-24
3,Chattisgarh,Kabirdham,Biranpur kalan (Sahaspur Lohra),Yellow,Oil Seeds,354,4100,4100,4100,10-Sep-24
4,Chattisgarh,Kabirdham,Kawardha,Yellow,Oil Seeds,0.1,4200,4200,4200,29-Aug-24


In [34]:
df['Reported Date'] = pd.to_datetime(df['Reported Date'], format='%d-%b-%y')


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18133 entries, 0 to 18132
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   State Name                 18133 non-null  object        
 1   District Name              18133 non-null  object        
 2   Market Name                18133 non-null  object        
 3   Variety                    18133 non-null  object        
 4   Group                      18133 non-null  object        
 5   Arrivals (Tonnes)          18124 non-null  object        
 6   Min Price (Rs./Quintal)    18133 non-null  int64         
 7   Max Price (Rs./Quintal)    18133 non-null  int64         
 8   Modal Price (Rs./Quintal)  18133 non-null  int64         
 9   Reported Date              18133 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 1.4+ MB


In [36]:
df.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date
0,Andhra Pradesh,Kurnool,Kurnool,Soyabeen,Oil Seeds,0.1,2509,4610,2509,2024-07-27
1,Chattisgarh,Balodabazar,Bhatapara,Other,Oil Seeds,0.4,4200,4281,4281,2024-09-26
2,Chattisgarh,Kabirdham,Biranpur kalan (Sahaspur Lohra),Yellow,Oil Seeds,340,4050,4050,4050,2024-08-02
3,Chattisgarh,Kabirdham,Biranpur kalan (Sahaspur Lohra),Yellow,Oil Seeds,354,4100,4100,4100,2024-09-10
4,Chattisgarh,Kabirdham,Kawardha,Yellow,Oil Seeds,0.1,4200,4200,4200,2024-08-29


In [37]:
from geopy.geocoders import ArcGIS

# Initialize ArcGIS geocoder with increased timeout
nom = ArcGIS(timeout=10)  # Set timeout to 10 seconds or more if needed
df["location"]=df.apply(lambda x: f"{x['State Name']}, {x['District Name']}, {x['Market Name']}",axis=1)

In [38]:
df_unique = df[['location']]
df_unique.drop_duplicates(inplace = True)

In [39]:
df_unique['latitude'] = df_unique["location"].apply(lambda x: nom.geocode(x).latitude)
df_unique['longitude'] = df_unique["location"].apply(lambda x: nom.geocode(x).longitude)

In [44]:
df_unique

,location,latitude,longitude
0,"Andhra Pradesh, Kurnool, Kurnool",15.831204,78.036177
1,"Chattisgarh, Balodabazar, Bhatapara",21.732599,81.946098
2,"Chattisgarh, Kabirdham, Biranpur kalan (Sahasp...",21.834106,81.126526
4,"Chattisgarh, Kabirdham, Kawardha",22.092511,81.222406
7,"Chattisgarh, Rajnandgaon, Gandai",21.665701,81.100403
...,...,...,...
16854,"Uttrakhand, UdhamSinghNagar, Bazpur",29.158001,79.147598
17015,"Telangana, Warangal, Warangal",17.974507,79.611362
17305,"Madhya Pradesh, Khandwa, Badwah(F&V)",21.821173,76.346161
17852,"Maharashtra, Jalana, Jafrabad",20.192411,76.019254


In [45]:
# @title Combining the dataset
# Merge df with df_unique on the 'location' column
df_merged = pd.merge(df, df_unique, on='location', how='left')

# Display the first few rows of the merged DataFrame
df_merged.head()

,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date,location,latitude,longitude
0,Andhra Pradesh,Kurnool,Kurnool,Soyabeen,Oil Seeds,0.1,2509,4610,2509,2024-07-27,"Andhra Pradesh, Kurnool, Kurnool",15.831204,78.036177
1,Chattisgarh,Balodabazar,Bhatapara,Other,Oil Seeds,0.4,4200,4281,4281,2024-09-26,"Chattisgarh, Balodabazar, Bhatapara",21.732599,81.946098
2,Chattisgarh,Kabirdham,Biranpur kalan (Sahaspur Lohra),Yellow,Oil Seeds,340,4050,4050,4050,2024-08-02,"Chattisgarh, Kabirdham, Biranpur kalan (Sahasp...",21.834106,81.126526
3,Chattisgarh,Kabirdham,Biranpur kalan (Sahaspur Lohra),Yellow,Oil Seeds,354,4100,4100,4100,2024-09-10,"Chattisgarh, Kabirdham, Biranpur kalan (Sahasp...",21.834106,81.126526
4,Chattisgarh,Kabirdham,Kawardha,Yellow,Oil Seeds,0.1,4200,4200,4200,2024-08-29,"Chattisgarh, Kabirdham, Kawardha",22.092511,81.222406


In [52]:
# Assuming df_merged is your DataFrame
df_merged['Arrivals (Tonnes)'] = pd.to_numeric(df_merged['Arrivals (Tonnes)'], errors='coerce')

# Grouping data by state and reported date
grouped_df = df_merged.groupby(['State Name', 'Reported Date']).agg({
    'Modal Price (Rs./Quintal)': 'mean',
    'Min Price (Rs./Quintal)': 'min',
    'Max Price (Rs./Quintal)': 'max'
}).reset_index()

# Create separate charts for each state
unique_states = grouped_df['State Name'].unique()

for state in unique_states:
    state_data = grouped_df[grouped_df['State Name'] == state]

    # Create area chart
    fig = px.area(state_data,
                  x='Reported Date',
                  y='Modal Price (Rs./Quintal)',
                  title=f'Modal Prices of Oil Seeds in {state} with Confidence Intervals',
                  labels={'Modal Price (Rs./Quintal)': 'Modal Price (Rs./Quintal)',
                          'Reported Date': 'Reported Date'})

    # Adding confidence intervals
    fig.add_scatter(x=state_data['Reported Date'],
                     y=state_data['Max Price (Rs./Quintal)'],
                     mode='lines',
                     name='Max Price',
                     fill='tonexty',
                     fillcolor='rgba(255, 0, 0, 0.2)')

    fig.add_scatter(x=state_data['Reported Date'],
                     y=state_data['Min Price (Rs./Quintal)'],
                     mode='lines',
                     name='Min Price',
                     fill='tonexty',
                     fillcolor='rgba(0, 255, 0, 0.2)')

    fig.show()

In [53]:
df_merged.describe()

,Arrivals (Tonnes),Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Reported Date,latitude,longitude
count,18124.000000,18133.000000,18133.000000,18133.000000,18133,18133.000000,18133.000000
mean,88.460575,3772.882976,4469.138587,4348.046821,2024-08-14 11:23:16.283019776,22.070387,76.471468
min,0.010000,0.000000,0.000000,2300.000000,2024-07-01 00:00:00,9.444085,69.448779
25%,5.500000,3500.000000,4260.000000,4160.000000,2024-07-21 00:00:00,20.684660,75.350700
50%,26.055000,4000.000000,4396.000000,4300.000000,2024-08-13 00:00:00,22.718673,76.433601
75%,94.000000,4200.000000,4525.000000,4423.000000,2024-09-09 00:00:00,23.786501,77.515240
max,4507.100000,9000.000000,45421.000000,9000.000000,2024-09-30 00:00:00,29.158001,93.992703
std,159.960970,958.214220,825.596461,619.114654,NaN,2.398615,2.397429


In [58]:
# Convert 'Reported Date' to datetime if not already done
df_merged['Reported Date'] = pd.to_datetime(df_merged['Reported Date'])

# Extract month and year from the 'Reported Date'
df_merged['Month'] = df_merged['Reported Date'].dt.to_period('M')

# Group by Month, District, and Market to calculate the required metrics
monthly_summary = df_merged.groupby(['State Name', 'District Name', 'Market Name', 'Month']).agg(
    Arrivals_Tonnes=('Arrivals (Tonnes)', 'sum'),
    Average_Price=('Modal Price (Rs./Quintal)', 'mean'),
    Max_Price=('Max Price (Rs./Quintal)', 'max'),
    Min_Price=('Min Price (Rs./Quintal)', 'min')
).reset_index()

# Convert 'Month' back to a string for better presentation
monthly_summary['Month'] = monthly_summary['Month'].astype(str)

# Display the summary for each state
for state in monthly_summary['State Name'].unique():
    print(f"Monthly Summary for {state}:")
    state_summary = monthly_summary[monthly_summary['State Name'] == state]
    print(state_summary)
    print("\n")


Monthly Summary for Andhra Pradesh:
       State Name District Name Market Name    Month  Arrivals_Tonnes  \
0  Andhra Pradesh       Kurnool     Kurnool  2024-07              2.1   
1  Andhra Pradesh       Kurnool     Kurnool  2024-08              1.1   

   Average_Price  Max_Price  Min_Price  
0        3814.25       4610       2509  
1        3164.00       4610       2869  


Monthly Summary for Chattisgarh:
     State Name District Name                      Market Name    Month  \
2   Chattisgarh   Balodabazar                        Bhatapara  2024-07   
3   Chattisgarh   Balodabazar                        Bhatapara  2024-08   
4   Chattisgarh   Balodabazar                        Bhatapara  2024-09   
5   Chattisgarh      Bemetara                             Saja  2024-07   
6   Chattisgarh      Bemetara                             Saja  2024-09   
7   Chattisgarh     Kabirdham  Biranpur kalan (Sahaspur Lohra)  2024-07   
8   Chattisgarh     Kabirdham  Biranpur kalan (Sahaspur Lohra

In [59]:
# Save the monthly_summary DataFrame to a CSV file
monthly_summary.to_csv('monthly_summary.csv', index=False)

print("Monthly summary dataset extracted to 'monthly_summary.csv'.")


Monthly summary dataset extracted to 'monthly_summary.csv'.


In [61]:
monthly_summary.head()

,State Name,District Name,Market Name,Month,Arrivals_Tonnes,Average_Price,Max_Price,Min_Price
0,Andhra Pradesh,Kurnool,Kurnool,2024-07,2.1,3814.25,4610,2509
1,Andhra Pradesh,Kurnool,Kurnool,2024-08,1.1,3164.00,4610,2869
2,Chattisgarh,Balodabazar,Bhatapara,2024-07,3.6,3696.40,3950,3300
3,Chattisgarh,Balodabazar,Bhatapara,2024-08,1.3,3350.00,3900,2300
4,Chattisgarh,Balodabazar,Bhatapara,2024-09,1.9,3340.00,4316,2399


In [62]:
monthly_summary.shape

(1265, 8)

# weekly report of Soya Bean

In [69]:
# Convert 'Reported Date' to datetime if not already done
df_merged['Reported Date'] = pd.to_datetime(df_merged['Reported Date'])

# Extract week from the 'Reported Date'
df_merged['Week'] = df_merged['Reported Date'].dt.to_period('W')

# Group by Week, District, and Market to calculate the required metrics
weekly_summary = df_merged.groupby(['State Name', 'District Name', 'Market Name', 'Week']).agg(
    Arrivals_Tonnes=('Arrivals (Tonnes)', 'sum'),
    Average_Price=('Modal Price (Rs./Quintal)', 'mean'),
    Max_Price=('Max Price (Rs./Quintal)', 'max'),
    Min_Price=('Min Price (Rs./Quintal)', 'min')
).reset_index()

# Convert 'Week' back to a string for better presentation
weekly_summary['Week'] = weekly_summary['Week'].astype(str)

# Display the summary for each state
for state in weekly_summary['State Name'].unique():
    print(f"Weekly Summary for {state}:")
    state_summary = weekly_summary[weekly_summary['State Name'] == state]
    print(state_summary)
    print("\n")


Weekly Summary for Andhra Pradesh:
       State Name District Name Market Name                   Week  \
0  Andhra Pradesh       Kurnool     Kurnool  2024-07-01/2024-07-07   
1  Andhra Pradesh       Kurnool     Kurnool  2024-07-22/2024-07-28   
2  Andhra Pradesh       Kurnool     Kurnool  2024-08-05/2024-08-11   

   Arrivals_Tonnes  Average_Price  Max_Price  Min_Price  
0              2.0    4249.333333       4600       4181  
1              0.1    2509.000000       4610       2509  
2              1.1    3164.000000       4610       2869  


Weekly Summary for Chattisgarh:
     State Name District Name  Market Name                   Week  \
3   Chattisgarh   Balodabazar    Bhatapara  2024-07-01/2024-07-07   
4   Chattisgarh   Balodabazar    Bhatapara  2024-07-15/2024-07-21   
5   Chattisgarh   Balodabazar    Bhatapara  2024-08-05/2024-08-11   
6   Chattisgarh   Balodabazar    Bhatapara  2024-08-12/2024-08-18   
7   Chattisgarh   Balodabazar    Bhatapara  2024-08-26/2024-09-01   
..  

In [70]:
weekly_summary.head()

,State Name,District Name,Market Name,Week,Arrivals_Tonnes,Average_Price,Max_Price,Min_Price
0,Andhra Pradesh,Kurnool,Kurnool,2024-07-01/2024-07-07,2.0,4249.333333,4600,4181
1,Andhra Pradesh,Kurnool,Kurnool,2024-07-22/2024-07-28,0.1,2509.000000,4610,2509
2,Andhra Pradesh,Kurnool,Kurnool,2024-08-05/2024-08-11,1.1,3164.000000,4610,2869
3,Chattisgarh,Balodabazar,Bhatapara,2024-07-01/2024-07-07,2.3,3945.500000,3950,3850
4,Chattisgarh,Balodabazar,Bhatapara,2024-07-15/2024-07-21,1.3,3530.333333,3791,3300


In [66]:
weekly_summary.shape

(4639, 8)

In [72]:
# Filter the summary for Nagpur
nagpur_summary = weekly_summary[weekly_summary['District Name'] == 'Nagpur']

# Display the summary for Nagpur
print("Weekly Summary for Nagpur:")
nagpur_summary.head()


Weekly Summary for Nagpur:


,State Name,District Name,Market Name,Week,Arrivals_Tonnes,Average_Price,Max_Price,Min_Price
3445,Maharashtra,Nagpur,Bhiwapur,2024-07-08/2024-07-14,14.2,3950.0,4300,3600
3446,Maharashtra,Nagpur,Bhiwapur,2024-07-22/2024-07-28,40.0,4062.0,4325,3800
3447,Maharashtra,Nagpur,Bhiwapur,2024-08-05/2024-08-11,20.0,4015.0,4205,3825
3448,Maharashtra,Nagpur,Bhiwapur,2024-08-19/2024-08-25,30.0,3837.0,4000,3675
3449,Maharashtra,Nagpur,Bhiwapur,2024-09-09/2024-09-15,14.0,4215.0,4325,4100
